# Init

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import kaiser

In [2]:
%matplotlib qt5

In [3]:
import sys
import os

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.ser as ser
sys.path.insert(0, "/home/pleroy/DATA/DIADEM/SIEPEL/SER")
import wr15_wr10_dates

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
cmap='jet'
withPlots = 0
center = 6500 # old value 6650
width = 500   # old value 500
withPlot = 0
idxElevation = 18 # elevation correspondant au maximum releve avec la plaque support
centerFreq = 92.5
bandWidth = 17.4
nb_elev = 41
nb_freq = 2801
nb_ssb  = 4
elevation = np.linspace( 0.25, 1.05, nb_elev )
applyCorrection = 1

**190911** B

**190912** F / G / AHP12 / C / D / J / E => Sonde d'illumination dans sa position par défaut (0° au lieu de -43.3°) pendant toutes les mesures.

**190913** A / E / J / Jb / D / C => Polar CATR -43.3°

**190916** AHP12 / G / M / L / R / U / U2 / K / I / H / O

Axe élévation du positionneur non initialisé, toutes les mesures réalisées avec la même élévation.
* possibilité d'utiliser les mesures du 190912 pour les absorbants AHP12 G
* refaire les mesures pour les absorbants M L R U K I H O

**190917** N / AHP45 / AHP30 / AHP9 / AHP30b

**190918** F

**190923** M / L / R / U_x / I_x / H_x => Mauvaise cal utilisée pour U_x, I_x et H_x

**190925** K_x / O_x => Mauvaise cal utilisée pour K_x et O_x

**Pour les mesures qui suivent: FI à -46.5MHz et step de 8.75MHz permettant d'éviter le point à 96GHz pendant la cal**

**190926** R

**190927** K / O / I / H / Support / A / E

**190930** J / D / C / AHP12 / G / F / B / U / AHP30

**191001** AHP9 / AHP45 / L / M / N / SupportMax

**191002** Plaque / PlaqueRef

**191004** Support_x / AHP12_x / Supportb_x / Supportc / AHP12b / AHP9 / AHP20 / AHP30 / AHP45

Pour les x, le fichier de séquence utilisé était celui de la plaque de référence et pas celui de la plaque support.

In [7]:
base_path = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR10"
processing_path = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR10_processing"

## Probe

In [8]:
tmp = np.loadtxt( f"{base_path}/190910/S11W_PROBE.s1p", skiprows=5)
S11_probe_a = ( 10 ** (0.05 * tmp[:,1]) ) * np.exp( 1j * np.pi / 180 * ( tmp[:,2] ) )
S11_probe_a = S11_probe_a.conj()
d = np.linspace( 0, 300 / ( tmp[1,0] - tmp[0,0] ), nb_freq ) * 0.5 * 1e9

In [11]:
tmp = np.loadtxt(f"{processing_path}/data/S11_probe_b.txt", skiprows=1)
S11_probe_b = tmp[:,1] + 1j * tmp[:,2]

## Fond de chambre

In [13]:
# THE DATA STORED AFTER PROCESSING ARE ALREADY CONJUGATED
tmp = np.loadtxt(f"{processing_path}/data/S11_fond_de_chambre_a.txt", skiprows=1)
S11_f2c_a = tmp[:,1] + 1j * tmp[:,2]
tmp = np.loadtxt(f"{processing_path}/data/S11_fond_de_chambre_b.txt", skiprows=1)
S11_f2c_b = tmp[:,1] + 1j * tmp[:,2]

In [14]:
if 1:
    fig, ax = plt.subplots(1,1)
    plt.plot( ser.dB( np.fft.ifft( S11_f2c_a ) ), label="fond de chambre a" )
    plt.plot( ser.dB( np.fft.ifft( S11_f2c_b ) ), label="fond de chambre b" )
    plt.grid()
    plt.legend()

## Plaque de référence

In [15]:
# THE DATA STORED AFTER PROCESSING ARE ALERADY CONJUGATED
tmp = np.loadtxt(f"{processing_path}/data/S11_plaque_ref_from_scan_a.txt", skiprows=1)
S11_plaque_a = tmp[:,1] + 1j * tmp[:,2]
tmp = np.loadtxt(f"{processing_path}/data/S11_plaque_ref_from_scan_b.txt", skiprows=1)
S11_plaque_b = tmp[:,1] + 1j * tmp[:,2]

In [16]:
if 0:
    fig, ax = plt.subplots(1,1)
    plt.plot( ser.dB( np.fft.ifft( S11_plaque_a ) ), label="plaque_a" )
    plt.plot( ser.dB( np.fft.ifft( S11_f2c_a ) ), label="fond de chambre a" )
    plt.plot( ser.dB( np.fft.ifft( S11_f2c_b ) ), label="fond de chambre b" )
    plt.grid()
    plt.legend()

In [17]:
if 1:
    fig, ax = plt.subplots(1,1)
    ax.plot( d, ser.dB( np.fft.ifft( S11_plaque_a ) ), label="plaque_a" )
    ax.plot( d, ser.dB( np.fft.ifft( S11_f2c_a ) ), label="fond de chambre a" )
    ax.plot( d, ser.dB( np.fft.ifft( S11_probe_a ) ), label="probe_a" )
    ax.plot( d, ser.dB( np.fft.ifft( S11_plaque_a - S11_probe_a ) ), label="plaque_a - probe_a" )
    ax.plot( d, ser.dB( np.fft.ifft( S11_plaque_a - S11_f2c_a ) ), label="plaque_a - fond de chambre a" )
    ax.plot( d, ser.dB( np.fft.ifft( S11_plaque_a - S11_f2c_b ) ), label="plaque_a - fond de chambre b" )
    ax.grid()
    ax.legend()
    ax.set_xlabel("[m]")
    ax.set_ylabel("[dB]")
    ax.set_title( "time domain" )

# Réflectivité
ref: ser4siepel_multiband_scan_elev_Reflectivite.sce

In [18]:
gate = ser.getWindow( d, center, width, id="llc" )

## Lecture des données

In [19]:
idx96GHz = 1680

In [20]:
def get_S11_FondDeChambre_S11_plaque( serie, f2cEN ):
    if serie == "a":
        if f2cEn:
            S11_fondDeChambre = S11_f2c_a
        else:
            S11_fondDeChambre = S11_probe_a
        S11_plaque = S11_plaque_a
    elif serie == "b":
        if f2cEn:
            S11_fondDeChambre = S11_f2c_b
        else:
            S11_fondDeChambre = S11_probe_b
        S11_plaque = S11_plaque_b
    else:
        print("specify a valid serie")
    return S11_fondDeChambre, S11_plaque

In [22]:
f2cEn = 0

if f2cEn:
    suffix = ""
else:
    suffix = "_pr"

serie, date, absorbant = wr15_wr10_dates.dictOfMeasurements["AHP12"][3]
data_dir = f"{base_path}/{date}/{absorbant}" 
freq_abs, S11 = ser.getData_ssb( nb_elev, nb_freq, nb_ssb, data_dir )
if serie == "a":
    S11_abs = S11.conj()
elif serie == "b":
    S11_abs = S11
else:
    print("specify a valid serie")
S11_fondDeChambre, S11_plaque = get_S11_FondDeChambre_S11_plaque( serie, f2cEn )

## Calcul de la réflectivité

In [ ]:
reflectivite = ser.getReflectivite( S11_abs, S11_plaque, S11_fondDeChambre, gate )

In [ ]:
toPlot = np.zeros( ( nb_elev, nb_freq ) )
gatebis  = 0.0 * gate
gatebis[ np.where( np.abs( freq_abs - centerFreq ) < bandWidth ) ] = 1.0
toPlot = 20 * np.log10( 1e-12 + np.abs( reflectivite * gatebis ) )

valMin = -90
toPlot = 0.5 * ( np.abs( toPlot - valMin ) + toPlot - valMin ) + valMin
valMax = np.amax( toPlot )
valMin = np.amin( toPlot )
print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")

fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( elevation, np.flip( freq_abs ), np.rot90(toPlot), cmap=cmap )
title = f"Réflectivité absorbant {absorbant}, {date}, série {serie}"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )

In [ ]:
descriptor = f"{absorbant}_{date}"
filename = f"{base_path}/processing/data_reflectivite{suffix}/reflectivite_{descriptor}{suffix}"
np.save( filename, reflectivite )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot( freq_abs, toPlot[ idxElevation, : ] )
ax.grid()
title = f"Réflectivité [{absorbant}]"
ax.set_title(title)
ax.set_xlabel("f [GHz]")
ax.set_ylabel("réflectivité [dB]")

In [ ]:
if 0:
    fig, ax = plt.subplots(1,1)
    ax.plot(gatebis)

In [ ]:
np.save( f"{base_path}/processing/data/freq_WR10", freq_abs )
np.save( f"{base_path}/processing/data/elevation_WR10", elevation )

# Tous les absorbants

In [26]:
idx = np.where( np.abs( freq_abs - centerFreq ) < bandWidth )
idxAlt = np.where( np.abs( freq_abs - centerFreq ) >= bandWidth )
vmin = -90
vmax = -40
dic_stats = {}
toPlot = np.zeros( ( nb_elev, nb_freq ) )
myAbs = "H"
myDict = {myAbs: wr15_wr10_dates.dictOfMeasurements[myAbs]}

f2cEn = 1

if f2cEn:
    suffix = ""
else:
    suffix = "_pr"

for item in wr15_wr10_dates.dictOfMeasurements.items(): # all => wr15_wr10_dates.dictOfMeasurements.items()
    ref, tests = item
    for test in tests:
        serie, date, absorbant = test
        data_dir = f"{base_path}/{date}/{absorbant}"
        descriptor = f"{absorbant}_{date}"
        if serie == "-":
            pass
        else:
            freq_abs, S11 = ser.getData_ssb( nb_elev, nb_freq, nb_ssb, data_dir )
            
            if serie == "a":
                S11_abs = S11.conj()
                print(f"{serie} === {date} === {absorbant}")
            elif serie == "b":
                S11_abs = S11
                print(f"{serie} === {date} === {absorbant}")
            else:
                print(f"{serie} === {date} === {absorbant} === NOT PROCESSED")
            S11_fondDeChambre, S11_plaque = get_S11_FondDeChambre_S11_plaque( serie, f2cEn )
            
            #=============
            # REFLECTIVITE
            reflectivite = ser.getReflectivite( S11_abs, S11_plaque, S11_fondDeChambre, gate )
            np.save( 
                f"{processing_path}/data/reflectivite{suffix}/{descriptor}{suffix}",
                reflectivite 
            )
    
            # drop lower and higher frequencies values
            toPlot[:,idx]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idx] ) )
            toPlot[:,idxAlt] = -90

            fig, ax = plt.subplots( 1, 1 )
            im = ax.pcolormesh( elevation, np.flip( freq_abs ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
            title = f"Réflectivité [{descriptor}]{suffix}"
            ax.set_title(title)
            ax.set_xlabel( "Elévation [°]" )
            ax.set_ylabel( "Fréquence [GHz]" )
            dia.addColorBar(  im, ax, aspect='auto' )
            fig.savefig(
                f"{processing_path}/plot/reflectivite{suffix}/reflectivite_{descriptor}{suffix}.png"
            )

            #======
            # STATS
            valMax = np.amax( toPlot )
            valMin = np.amin( toPlot )

            absReflectivite   = np.abs( reflectivite )
            reflectiviteMoy   = np.average( absReflectivite[:,idx] )
            reflectiviteSigma = np.std( absReflectivite[:,idx] )

            reflectiviteMoy_dB   = 20 * np.log10( np.abs( reflectiviteMoy ) )
            reflectiviteSigma_dB = 20 * np.log10( np.abs( reflectiviteSigma ) )
        
            #==========
            # HISTOGRAM
            fig, ax = plt.subplots(1,1)
            ax.hist((absReflectivite[:,idx]).flatten(), 50, label="histogramme (valeurs linéaires)")
            ax.grid()
            title = f"Réflectivité, histogramme [{descriptor}]{suffix}"
            ax.set_title(title)
            ax.legend()
            fig.savefig(
                f"{processing_path}/plot/hist{suffix}/hist_{descriptor}{suffix}.png"
            )
        
            #===========
            # PLOT WRT F
            fig, ax = plt.subplots(1,1)
            ax.plot( freq_abs, toPlot[ idxElevation, : ] )
            ax.grid()
            title = f"Réflectivité [{descriptor}]{suffix}"
            ax.set_title(title)
            ax.set_xlabel("f [GHz]")
            ax.set_ylabel("réflectivité [dB]")
            fig.savefig(
                f"{processing_path}/plot/reflectivite_vs_f{suffix}/reflectivite_vs_f_{descriptor}{suffix}.png"
            )
    
            plt.close("all")
    
            dic_stats[f"{descriptor}"] = (valMax, valMin, reflectiviteMoy, reflectiviteSigma, reflectiviteMoy_dB)

a === 190913 === A
b === 190927 === A
a === 190911 === B
b === 190930 === B
a === 190912 === C
a === 190913 === C
b === 190930 === C
a === 190912 === D
a === 190913 === D
b === 190930 === D
a === 190912 === E
a === 190913 === E
b === 190927 === E
a === 190912 === F
a === 190918 === F
b === 190930 === F
a === 190912 === G
b === 190930 === G
b === 190927 === H
b === 190927 === I
a === 190912 === J
a === 190913 === J
a === 190913 === Jb
b === 190930 === J
b === 190927 === K
a === 190923 === L
b === 191001 === L
a === 190923 === M
b === 191001 === M
a === 190917 === N
b === 191001 === N
b === 190927 === O
b === 190930 === U
a === 190917 === AHP9
b === 191001 === AHP9
b === 191004 === AHP9
a === 190912 === AHP12
b === 190930 === AHP12
b === 191004 === AHP12b
a === 190923 === R
b === 190926 === R
b === 191004 === AHP20
a === 190917 === AHP30
a === 190917 === AHP30b
b === 190930 === AHP30
b === 191004 === AHP30
a === 190917 === AHP45
b === 191001 === AHP45
b === 191004 === AHP45


In [27]:
for absorbant in dic_stats.keys():
    stats = dic_stats[absorbant]
    print(f"{absorbant} {stats[0]:.2f} {stats[1]:.2f} {stats[2]:.6f} {stats[3]:.6f} {stats[4]:.2f}")

A_190913 -41.04 -107.96 0.002103 0.001727 -53.54
A_190927 -39.22 -107.97 0.002174 0.001887 -53.26
B_190911 -52.80 -111.76 0.000458 0.000263 -66.79
B_190930 -52.44 -122.23 0.000497 0.000262 -66.08
C_190912 -53.92 -117.26 0.000681 0.000420 -63.34
C_190913 -52.96 -110.93 0.000794 0.000404 -62.01
C_190930 -49.38 -106.13 0.001140 0.000486 -58.86
D_190912 -56.05 -120.12 0.000312 0.000173 -70.12
D_190913 -55.54 -120.31 0.000349 0.000212 -69.15
D_190930 -52.65 -124.11 0.000349 0.000225 -69.13
E_190912 -55.60 -115.58 0.000391 0.000248 -68.15
E_190913 -53.98 -112.85 0.000557 0.000349 -65.09
E_190927 -53.30 -116.91 0.000596 0.000333 -64.50
F_190912 -55.56 -117.38 0.000447 0.000216 -67.00
F_190918 -48.59 -115.14 0.000432 0.000229 -67.29
F_190930 -53.53 -120.34 0.000359 0.000177 -68.91
G_190912 -53.44 -118.49 0.000459 0.000264 -66.77
G_190930 -52.11 -114.77 0.000533 0.000242 -65.46
H_190927 -52.07 -121.96 0.000657 0.000546 -63.65
I_190927 -52.99 -114.53 0.000643 0.000366 -63.84
J_190912 -55.08 -119

# Remove all

In [ ]:
for root, dirs, files in os.walk(f"{base_path}/processing"):
    for f in files:
        if f.endswith(".png") or f.endswith(".npy"):
            os.remove(os.path.join(root, f)) # remove or unlink does the same

In [ ]:
os.walk(f"{base_path}/processing")

# getAzSlRoElPolFFPolCATR

In [ ]:
el = ser.getAzSlRoElPolFFPolCATR( "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR10/190910/Plaque" )[:,3]
print(f"elevation goes from {np.amin(el)} to {np.amax(el)}")